# Генерация описания для картин с помощью модели Salesforce/blip-image-captioning-large

In [ ]:
!pip install transformers

In [ ]:
# Импорт необходимых библиотек

import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import os
import pandas as pd
import gspread

In [ ]:
# создаем экземпляр класса из модели Salesforce/blip-image-captioning-large для обработки изображений

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [ ]:
# Создание пустого DataFrame, в который будем записывать описание к каждому файлу картины

df = pd.DataFrame(columns=["Название файла", "Описание картины"])
dataset_path = "/content/drive/MyDrive/2"

In [ ]:
# Изображение открывается, обрабатывается моделью, и генерируется описание картины. 
# Полученные данные сохраняются в DataFrame для дальнейшей обработки.

for author_folder in os.listdir(dataset_path):
    author_path = os.path.join(dataset_path, author_folder)

    for image_file in os.listdir(author_path):
        image_path = os.path.join(author_path, image_file)
        raw_image = Image.open(image_path).convert('RGB')
        inputs = processor(raw_image, return_tensors="pt").to("cuda")
        out = model.generate(**inputs, max_new_tokens=100)  # Используем max_new_tokens вместо max_length
        description = processor.decode(out[0], skip_special_tokens=True)
        temp_df = pd.DataFrame({"Название файла": [image_file], "Описание картины": [description]})
        df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
# Путь к файлу для сохранения в формате CSV
output_file = "/content/drive/MyDrive/описание_2.csv"

# Сохранение DataFrame в файл CSV
df.to_csv(output_file, index=False)

NameError: ignored

In [ ]:
df

,Название файла,Описание картины
0,houses-of-parlilament-sunlight-effect-1901.jpg,painting of a painting of a castle in the dist...
1,antibes-afternoon-effect(1).jpg,a painting of a castle on a rocky island with ...
2,girls-in-the-garden.jpg,a painting of two women in a garden with flowers
3,mount-kolsaas-in-misty-weather.jpg,painting of a mountain with snow on it and a s...
4,lunch-on-the-grass-study.jpg,painting of a man and woman in a park with trees
...,...,...
2977,the-pont-saint-michel-1900.jpg,painting of a train station with a bridge and ...
2978,memory-of-oceania.jpg,painting of a person holding a knife and a kni...
2979,portrait-of-andre-derain-1905.jpg,a painting of a man with a mustache and a red hat
2980,self-portrait-1900.jpg,painting of a man with a beard and a white shirt
